# Feedforward and feedback interactions for sequence learning in the macaque frontotemporal network

## Initialise and setup workspace
In this section, we will set up the workspace required for data analysis and visualization. The code is organized into several key steps:

### Importing Toolboxes and Dependencies
We start by importing essential Python libraries such as pandas for data manipulation, numpy for numerical operations, and matplotlib for plotting.

In [ ]:
### Import toolboxes and dependencies -----------------------------------------------------#
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

#------------------------------------------------------------------------------------------#
warnings.filterwarnings('ignore')


### Adding Directories to the System Path
To ensure that Python can locate all necessary modules, we add the root directory and its subdirectories to the system path. This allows us to seamlessly import custom functions and modules from different locations within the project.

In [ ]:
### Add root & subdirectories to path -----------------------------------------------------#
def init_path(root_dir):
    """
    Recursively add a directory and all its subdirectories to the system path.
    """
    for dirpath, dirnames, filenames in os.walk(root_dir):
        if dirpath not in sys.path:
            sys.path.append(dirpath)
                  
          
if sys.platform == 'win32':  
    init_path(r'C:\KIKUCHI-LOCAL\script\2024-aglt-laminar')
elif sys.platform == 'darwin':
    init_path(r'/Users/stevenerrington/Desktop/Projects/2024-aglt-laminar')  
#------------------------------------------------------------------------------------------#

### Importing Custom Functions
We import custom functions from various modules specific to our project. These functions include setup procedures, MATLAB-related functions, spike analysis tools, behavioral analysis utilities, and figure generation tools.

In [ ]:
### Import custom functions ---------------------- ----------------------------------------#
from setup_workspace import *
from matlab_functions import *
from nphys_functions import *
from beh_functions import *
from figure_functions import *
#------------------------------------------------------------------------------------------#


### Loading Experiment Logs and Defining Data Directories
Finally, we import and clean the experimental logs necessary for the analysis. This step includes setting up directories for data storage and retrieval, and ensuring that the logs are in a suitable format for further processing.

In [ ]:
### Import experiment logs and define data directories ------------------------------------#
dirs = set_directories()                                    # Get directories
ephysLog, stimulusLog, spike_log = import_exp_map()         # Get experimental logs
ephysLog = clean_exp_map(ephysLog)                          # Clean experiment log
spike_log = clean_spike_map(spike_log)                      # Clean spike log
#------------------------------------------------------------------------------------------#

By following these steps, we prepare the environment for subsequent data analysis tasks, ensuring that all dependencies and custom functions are correctly configured.

## Functional properties of auditory and frontal cortex neurons to auditory sequences

### Identify modulatory activity of neurons across different epochs

To identify the activity of neurons in response to different task-events, we will align the spiking activity of isolated neurons to the trial start cue, the sequence onset, and the reward modulation.

Here we define parameters important for this analysis, and identify the area from which neurons are recorded based on their label in the recording log.

In [ ]:
import numpy as np

# Set parameters
ops = {
    'timewin': np.arange(-1000, 5001, 1),  # -1000 to 5000 inclusive
    'bl_win': np.arange(-150, -49, 1),     # -150 to -50 inclusive
    'freq': [2, 60],                       # Frequencies range from 2 to 60
    'sdf_filter': 'PSP',
    'sig_threshold': 2,
    'min_sig_time': 50
}

# Set parameters for sound specific stimuli
ops['sound_sdf_window'] = np.arange(-200, 601, 1)  # -200 to 600 inclusive
sound_onset_ms = [0, 563, 1126, 1689, 2252]
zero_offset = abs(ops['timewin'][0])

# Define neurons
auditory_neuron_idx = np.where(np.isin(spike_log['area'], ['R', 'A1', 'RM', 'dSTS']))[0]
frontal_neuron_idx = np.where(np.isin(spike_log['area'], ['44', '45', 'FOP']))[0]

# Initialize loop variables
neuron_count = 0
neuron_sdftrialstart_out = []
neuron_sdfviol_out = []
neuron_sdfseq_out = []
neuron_sdfreward_out = []
neuron_sdfsound_out = []
neuron_baseline_out = []

In [ ]:
for session_i in range(ephysLog.shape[0]):
    datafile = ephysLog.loc[session_i, 'session']
    print(f'Session {session_i + 1} of {ephysLog.shape[0]} | {datafile}')
    
    # Load event table data
    event_table = pd.read_csv(os.path.join(dirs["py_data"], 'event_table', datafile + '-events.csv'))        

    # Adjust onsets based on audio latency measures
    session_audio_latency = scipy.io.loadmat(r'C:\KIKUCHI-LOCAL\script\2024-aglt-laminar\data-extraction\doc\session_audio_latency.mat')
    session_audio_latency = session_audio_latency['session_audio_latency']
    event_table['stimulusOnset_ms'] += session_audio_latency[session_i][0][:,0]

    # Create a violation time event for alignment
    create_violation_alignment_event(event_table)

    # Find all neurons in the given session
    neuron_list = spike_log['unitDSP'][spike_log['session'] == datafile]
    
    # For each neuron recorded in the session    
    for neuron in neuron_list:
        # Update the loop variables
        neuron_count += 1
        sound_sdf = []
        count = 0    
    
        # Load the spike times from the datafile
        spike_times = np.loadtxt(os.path.join(dirs["py_data"], 'spike_times', datafile + '-' + neuron + '-spk.txt'), delimiter=',')
        spike_times = spike_times.astype(int)
        
         # Generate a spike density function
        sdf = spk_convolve(spike_times, np.max(spike_times)+10000, 'PSP')

        # Aligned spike density functions and rasters    
        align_event = 'trialStart_ms'
        align_times = event_table[align_event].to_numpy(dtype=int)
        sdf_aligned_trialStart, raster_aligned_trialStart = spk_align(sdf, spike_times, align_times, ops['timewin'])

        align_event = 'stimulusOnset_ms'
        align_times = event_table[align_event].to_numpy(dtype=int)
        sdf_aligned_seq, raster_aligned_seq = spk_align(sdf, spike_times, align_times, ops['timewin'])

        align_event = 'violation_ms'
        align_times = event_table[align_event].to_numpy(dtype=int)
        sdf_aligned_viol, raster_aligned_viol = spk_align(sdf, spike_times, align_times, ops['timewin'])

        align_event = 'rewardOnset_ms'
        align_times = event_table[align_event].to_numpy(dtype=int)
        sdf_aligned_reward, raster_aligned_reward = spk_align(sdf, spike_times, align_times, ops['timewin'])
        
        # Store SDFs for different alignment events
        neuron_sdftrialstart_out.append(sdf_aligned_trialStart);
        neuron_sdfseq_out.append(sdf_aligned_seq);
        neuron_sdfviol_out.append(sdf_aligned_viol);
        neuron_sdfreward_out.append(sdf_aligned_reward);

        # Select data for the baseline window
        baseline_data = sdf_aligned_trialStart[:, zero_offset + ops['bl_win']]
        neuron_baseline = np.nanmean(baseline_data, axis=1)
        neuron_baseline_out.append(neuron_baseline)
        
        # Process each trial for the current neuron
        for trial_i in range(len(event_table)):
            if event_table.loc[trial_i, 'cond_label'] != 'error':  # Skip error trials
                for sound_i in range(5):  # Loop through each sound in the trial
                    count += 1
                    
                    # Extract and store relevant SDF information for each sound
                    sdf_value = sdf_aligned_seq[trial_i, zero_offset + ops['sound_sdf_window'] + sound_onset_ms[sound_i]]
                    raster_value = raster_aligned_seq[trial_i] - sound_onset_ms[sound_i]
                    
                    # Ensure `cond_value` is a valid key
                    cond_value = event_table.loc[trial_i, 'cond_value']
                    if cond_value in stimulusLog[f'sound_{sound_i + 1}_code'].index:
                        sound_code = stimulusLog[f'sound_{sound_i + 1}_code'][cond_value]
                    else:
                        sound_code = 'Unknown'  # Handle missing keys

                    sound_sdf.append([
                        sdf_value,
                        raster_value,
                        sound_code,
                        f'position_{sound_i + 1}',
                        event_table.loc[trial_i, 'cond_label']
                    ])
                    
        neuron_sdfsound_out.append(sound_sdf)
        


In [ ]:
# Assuming 'spike_log' is a DataFrame and other variables are already defined
# Initialize dictionaries or lists to store results
norm_fr_soundA = []
norm_fr_soundC = []
norm_fr_soundG = []
norm_fr_soundF = []
norm_fr_soundD = []
norm_fr_sound_all = []
norm_fr_sequence = []
norm_fr_trialStart = []
norm_fr_violation = []
norm_fr_consistant = []

# Normalize activity across sounds
for neuron_i in range(len(spike_log)):
    print(f'Normalizing activity for neuron {neuron_i + 1} of {len(spike_log)}...')

    # Identify trials for each sound type (A, C, G, F, D)
    sound_types = ['A', 'C', 'G', 'F', 'D']
    trial_indices = {sound: np.array([sdf[2] == sound for sdf in neuron_sdfsound_out[neuron_i]]) 
                     for sound in sound_types}
    
    # Calculate baseline firing rate statistics
    baseline_fr_mu = np.nanmean(neuron_baseline[neuron_i])
    baseline_fr_std = np.nanstd(neuron_baseline[neuron_i])

    print(baseline_fr_mu)



Normalizing activity for neuron 1 of 2298...
Normalizing activity for neuron 2 of 2298...
Normalizing activity for neuron 3 of 2298...
Normalizing activity for neuron 4 of 2298...
Normalizing activity for neuron 5 of 2298...
Normalizing activity for neuron 6 of 2298...
Normalizing activity for neuron 7 of 2298...
Normalizing activity for neuron 8 of 2298...
Normalizing activity for neuron 9 of 2298...
Normalizing activity for neuron 10 of 2298...
Normalizing activity for neuron 11 of 2298...
Normalizing activity for neuron 12 of 2298...
Normalizing activity for neuron 13 of 2298...
Normalizing activity for neuron 14 of 2298...
Normalizing activity for neuron 15 of 2298...
Normalizing activity for neuron 16 of 2298...
Normalizing activity for neuron 17 of 2298...
Normalizing activity for neuron 18 of 2298...
Normalizing activity for neuron 19 of 2298...
Normalizing activity for neuron 20 of 2298...
Normalizing activity for neuron 21 of 2298...
Normalizing activity for neuron 22 of 2298.

IndexError: index 129 is out of bounds for axis 0 with size 129

In [ ]:

    # Stack and concatenate sound-aligned SDF
    sound_sdf_temp = np.vstack([sdf[0] for sdf in neuron_sdfsound_out[neuron_i]])
    
    # Calculate baseline firing rate for the sound-aligned SDF
    baseline_window = slice(200 + (-100), 200 + 0)  # 100 ms before sound onset
    sound_baseline_fr_mu = np.nanmean(np.nanmean(sound_sdf_temp[:, baseline_window], axis=1))
    sound_baseline_fr_std = np.nanstd(np.nanmean(sound_sdf_temp[:, baseline_window], axis=1))
    
    # Normalize firing rates for each sound type
    norm_fr = {}
    for sound in sound_types:
        trials = trial_indices[sound]
        sound_sdf = np.vstack([sdf[0] for i, sdf in enumerate(neuron_sdfsound_out[neuron_i]) if trials[i]])
        norm_fr[f'sound{sound}'] = (np.nanmean(sound_sdf, axis=0) - sound_baseline_fr_mu) / sound_baseline_fr_std
    
    # Normalize firing rates across all sound trials
    all_sdf = np.vstack([sdf[0] for sdf in neuron_sdfsound_out[neuron_i]])
    norm_fr_sound_all.append((np.nanmean(all_sdf, axis=0) - baseline_fr_mu) / baseline_fr_std)
    
    # Normalize firing rates for sequence and trial start
    norm_fr_sequence.append((np.nanmean(neuron_sdfseq_out[neuron_i], axis=0) - baseline_fr_mu) / baseline_fr_std)
    norm_fr_trialStart.append((np.nanmean(neuron_sdftrialstart_out[neuron_i], axis=0) - baseline_fr_mu) / baseline_fr_std)

# Load event tables and normalize firing rates for violation and consistent trials
for neuron_i in range(len(spike_log)):
    datafile = spike_log.session.iloc[neuron_i]
    # Load event table data
    event_table = pd.read_csv(os.path.join(dirs["py_data"], 'event_table', datafile + '-events.csv'))        

    # Identify violation and consistent trials
    viol_trials = event_table.index[event_table['cond_label'] == 'viol'].tolist()
    cons_trials = event_table.index[event_table['cond_label'] == 'nonviol'].tolist()
    
    # Extract and store normalized firing rates for violation and consistent trials
    norm_fr_violation.append(neuron_sdfviol_out[neuron_i][viol_trials, :])
    norm_fr_consistant.append(neuron_sdfviol_out[neuron_i][cons_trials, :])

In [ ]:
## Spiking activity
# Determine neurons that have significant onset cue modulation


# Determine neurons that have significant sequence modulation
# Determine neurons that have significant reward modulation

# Determine neurons that have significant violation modulation

# Determine neurons that have theta modulation

## Laminar dynamics during auditory sequences

In [ ]:
# Spectrolaminar profile of auditory and frontal cortex recordings 
# Current source density plots (sound onset; auditory cortex)
# Current source density plots (reward delivery; frontal cortex)

## Fronto-temporal interactions during auditory sequences

In [ ]:
# 